In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
sns.set(style="whitegrid", palette="viridis")

df = pd.read_csv("../data/raw/housing.csv")

df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [ ]:
df.rename(columns={
    'longitude': 'longitud',
    'latitude': 'latitud',
    'housing_median_age': 'edad_media_vivienda',
    'total_rooms': 'total_habitaciones',
    'total_bedrooms': 'total_dormitorios',
    'population': 'poblacion',
    'households': 'hogares',
    'median_income': 'ingreso_medio',
    'median_house_value': 'valor_medio_vivienda',
    'ocean_proximity': 'proximidad_oceano'
}, inplace=True)

df.head()

In [ ]:
# Mapeo de nombres originales a nombres en español
mapa_proximidad = {
    "NEAR BAY": "Cerca de la bahía",
    "<1H OCEAN": "A menos de 1 hora del océano",
    "INLAND": "Tierra adentro",
    "NEAR OCEAN": "Cerca del océano",
    "ISLAND": "Isla"
}

# Aplicar el cambio
df["proximidad_oceano"] = df["proximidad_oceano"].replace(mapa_proximidad)

# Verificar resultado
df["proximidad_oceano"].value_counts()


In [ ]:
import matplotlib.pyplot as plt

df["proximidad_oceano"].value_counts().plot(
    kind="bar", color="skyblue", title="Distribución de proximidad al océano"
)
plt.xlabel("Categoría de proximidad")
plt.ylabel("Cantidad de zonas")
plt.show()


In [ ]:
df.isnull().sum()

In [ ]:
df["total_dormitorios"].fillna(df["total_dormitorios"].median(), inplace=True)
df.isnull().sum()


In [50]:
df["habitaciones_por_hogar"] = df["total_habitaciones"] / df["hogares"]
df["dormitorios_por_hogar"] = df["total_habitaciones"] / df["hogares"]
df["habitaciones_por_persona"] = df["total_habitaciones"] / df["poblacion"]

In [52]:
df = df[df["valor_medio_vivienda"] < 500000]

In [ ]:
df.info()
df.describe()

In [ ]:
df.hist(figsize=(12,8), bins=30, color='skyblue', edgecolor='black')
plt.suptitle("Distribucion de las variables del dataset", fontsize=16)
plt.show()


| Variable                 | Descripción                                                             | Tipo de dato | Observaciones                                  |
| ------------------------ | ----------------------------------------------------------------------- | ------------ | ---------------------------------------------- |
| **longitud**             | Coordenada geográfica (negativa por estar en el hemisferio occidental). | float        | Rango entre -124.35 y -114.31.                 |
| **latitud**              | Coordenada geográfica (posición norte-sur).                             | float        | Rango entre 32.54 y 41.95.                     |
| **edad_media_vivienda**  | Edad promedio de las viviendas.                                         | float        | Promedio: 28.6 años, máx.: 52.                 |
| **total_habitaciones**   | Número total de habitaciones por zona.                                  | float        | Promedio: 2.635, máx.: 39.320 (outliers).      |
| **total_dormitorios**    | Total de dormitorios por zona.                                          | float        | Algunos valores faltantes.                     |
| **poblacion**            | Número total de habitantes en la zona.                                  | float        | Promedio: 1.425, máx.: 35.682 (outliers).      |
| **hogares**              | Total de hogares por zona.                                              | float        | Promedio: 499 hogares.                         |
| **ingreso_medio**        | Ingreso medio expresado en decenas de miles de dólares.                 | float        | Promedio: 3.87 → ≈ USD 38.700 anuales.         |
| **valor_medio_vivienda** | Valor promedio de las viviendas (en dólares).                           | float        | Promedio: USD 206.855. Límite máximo: 500.001. |
| **proximidad_oceano**    | Categoría geográfica según cercanía al mar.                             | object       | Variable categórica.                           |


In [ ]:
sns.pairplot(df.sample(500), diag_kind="kde")
plt.suptitle("Relaciones entre variables principales", y=1.02)
plt.show()


In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Mapa de Correlación entre Variables Numéricas")
plt.show()


In [ ]:

columnas_numericas = df.select_dtypes(include=["float64", "int64"]).columns

for col in columnas_numericas:
    plt.figure(figsize=(6,4))
    sns.histplot(df[col], kde=True, color="skyblue")
    plt.title(f"Distribución de {col}")
    plt.show()


In [ ]:
df.info()
df.describe()


In [ ]:
correlaciones = df.corr(numeric_only=True)["valor_medio_vivienda"].sort_values(ascending=False)
correlaciones


In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x="ingreso_medio", y="valor_medio_vivienda", data=df, alpha=0.5)
plt.title("Relacion entre ingreso medio y valor medio de vivienda")
plt.xlabel("Ingreso medio (USD)")
plt.ylabel("Valor medio de vivienda (USD)")
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x="longitud", y="latitud",
    hue="valor_medio_vivienda", palette="coolwarm",
    data=df, alpha=0.5
)

plt.title("Distribucion geografica del valor medio de vivienda en California")
plt.show()

In [ ]:
sns.boxplot(x="proximidad_oceano", y="valor_medio_vivienda", data=df)
plt.title("Valor medio de vivienda segun proximidad al oceano")
plt.xticks(rotation=30)
plt.show()